<a href="https://colab.research.google.com/github/bebyakinb/test_assignments_Analyst/blob/master/habr/SQL_Hardest_Interview_Tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.Подготовка среды

In [1]:
!wget https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release -O sqlite.tar.gz &> /dev/null
!tar xzf sqlite.tar.gz &> /dev/null
%cd sqlite/
!./configure &> /dev/null
!make sqlite3.c &> /dev/null
%cd /content
!npx degit coleifer/pysqlite3 -f &> /dev/null
!cp sqlite/sqlite3.[ch] .
!python setup.py build_static build &> /dev/null
!cp build/lib.linux-x86_64-3.8/pysqlite3/_sqlite3.cpython-38-x86_64-linux-gnu.so \
    /usr/lib/python3.8/lib-dynload/_sqlite3.cpython-38-x86_64-linux-gnu.so

/content/sqlite
/content


In [1]:
import sqlite3
sqlite3.sqlite_version

'3.40.0'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Задача №1. Процентное изменение месяц к месяцу


## 1.0. Постановка задачи
**Контекст:** часто полезно знать, как изменяется ключевая метрика, например, месячная аудитория активных пользователей, от месяца к месяцу. Допустим у нас есть таблица logins в таком виде:
```
| user_id | date       |
|---------|------------|
| 1       | 2018-07-01 |
| 234     | 2018-07-02 |
| 3       | 2018-07-02 |
| 1       | 2018-07-02 |
| ...     | ...        |
| 234     | 2018-10-04 |
```
**Задача:** Найти ежемесячное процентное изменение месячной аудитории активных пользователей (MAU).

## 1.1. Подготовка данных

In [3]:
def random_dates(start, end, n=10):
    '''
    Dividing the unix time values(start and end) by 24*60*60*10**9
    to make it days and return n randomly chosen days back.
    '''
    start_u = start.value//(24*60*60*10**9)
    end_u = end.value//(24*60*60*10**9)

    return pd.to_datetime(np.random.randint(start_u, end_u, n), unit='D')


In [4]:
start = pd.to_datetime('2021-01-01')
end = pd.to_datetime('2022-12-01')
login_lines = 100000
user_ids = 30000

logins = pd.DataFrame()
logins['user_id'] = np.random.randint(1, user_ids, login_lines)
logins['date'] = pd.Series(random_dates(start, end, login_lines))
logins

,user_id,date
0,17017,2021-05-21
1,14463,2022-03-08
2,2059,2021-05-18
3,11139,2021-11-03
4,17911,2021-08-04
...,...,...
99995,2113,2022-02-17
99996,1904,2021-01-29
99997,3243,2021-09-06
99998,23381,2021-08-21


In [5]:
con = sqlite3.connect('db')
cur = con.cursor()

In [6]:
logins.to_sql('logins', con, if_exists='replace',index=False)

In [7]:
def select(sql):
  return pd.read_sql(sql,con)

In [8]:
sql = '''select * from logins'''
select(sql)

,user_id,date
0,17017,2021-05-21 00:00:00
1,14463,2022-03-08 00:00:00
2,2059,2021-05-18 00:00:00
3,11139,2021-11-03 00:00:00
4,17911,2021-08-04 00:00:00
...,...,...
99995,2113,2022-02-17 00:00:00
99996,1904,2021-01-29 00:00:00
99997,3243,2021-09-06 00:00:00
99998,23381,2021-08-21 00:00:00


## 1.2. Решение SQL

In [9]:
sql ='''
with months_active as (
  select strftime('%Y %m', date) as year_month,
         cast(count(user_id) as real) as users from logins
  group by year_month)

select 
year_month, users, (users - LAG(users) over(order by year_month))*100/LAG(users) over(order by year_month) MAU_percent
from months_active
'''
select(sql)

,year_month,users,MAU_percent
0,2021 01,4419.0,NaN
1,2021 02,3986.0,-9.798597
2,2021 03,4502.0,12.945309
3,2021 04,4295.0,-4.597956
4,2021 05,4260.0,-0.814901
5,2021 06,4305.0,1.056338
6,2021 07,4522.0,5.040650
7,2021 08,4427.0,-2.100840
8,2021 09,4436.0,0.203298
9,2021 10,4395.0,-0.924256


## 1.3 Решение Pandas

In [10]:
pt = logins.pivot_table(index=[(logins['date'].dt.to_period('M'))], 
                        values=['user_id'],
                        aggfunc="count"
                       )
pt['MAU,%mm'] = (pt['user_id'] - pt['user_id'].shift(1))/pt['user_id'].shift(1)*100
pt

,user_id,"MAU,%mm"
date,,
2021-01,4419,NaN
2021-02,3986,-9.798597
2021-03,4502,12.945309
2021-04,4295,-4.597956
2021-05,4260,-0.814901
2021-06,4305,1.056338
2021-07,4522,5.040650
2021-08,4427,-2.100840
2021-09,4436,0.203298


# 2.Задача №2. Маркировка древовидной структуры

## 2.0 Постановка задачи
Контекст: предположим, у вас есть таблица tree с двумя столбцами: в первом указаны узлы, а во втором — родительские узлы.
```
node   parent
1       2
2       5
3       5
4       3
5       NULL 
```
Задача: написать SQL таким образом, чтобы мы обозначили каждый узел как внутренний (inner), корневой (root) или конечный узел/лист (leaf), так что для вышеперечисленных значений получится следующее:
```
node    label  
1       Leaf
2       Inner
3       Inner
4       Leaf
5       Root
```

## 2.1 Подготовка данных

In [11]:
nodes = pd.DataFrame({'node': [1,2,3,4,5],
                      'parent':[2,5,5,3,np.nan]})
nodes

,node,parent
0,1,2.0
1,2,5.0
2,3,5.0
3,4,3.0
4,5,NaN


In [12]:
nodes.to_sql('nodes', con, if_exists='replace',index=False)

### 2.2 Решение SQL

In [13]:
sql = '''
select node,
       case
            when parent is null then 'Root'
            when node in (select parent from nodes) then 'Inner'
            else 'Leaf'
       end as label
from nodes
'''
select(sql)

,node,label
0,1,Leaf
1,2,Inner
2,3,Inner
3,4,Leaf
4,5,Root


## 2.3 Решение Pandas

In [14]:
nodes['label'] =  np.where(nodes['parent'].isna(), 'Root',
                  np.where(nodes['node'].isin(nodes['parent']), 'Inner', 'Leaf'))
nodes[['node','label']]

,node,label
0,1,Leaf
1,2,Inner
2,3,Inner
3,4,Leaf
4,5,Root


# 3. Задача № 3. Удержание пользователей в месяц.

## 3.0. Постановка задачи

**Контекст:** допустим, у нас есть статистика по авторизации пользователей на сайте в таблице logins:
```
| user_id | date       |
|---------|------------|
| 1       | 2018-07-01 |
| 234     | 2018-07-02 |
| 3       | 2018-07-02 |
| 1       | 2018-07-02 |
| ...     | ...        |
| 234     | 2018-10-04 |
```

**Задача:** написать запрос, который получает количество удержанных пользователей в месяц. В нашем случае данный параметр определяется как количество пользователей, которые авторизовались в системе и в этом, и в предыдущем месяце.


## 3.1. Подготовка данных

In [15]:
logins.to_sql('logins', con, if_exists='replace',index=False)

## 3.2. Решение SQL

In [16]:
sql = '''
with months_logins as (
  select distinct user_id, 
         strftime('%Y %m', date) as cur_month,
         strftime('%Y %m', date(date, 'start of month','-1 month')) as prev_month
  from logins
  )

select m1.cur_month, count(m2.user_id)
from months_logins as m1
left join months_logins as m2 ON m1.user_id = m2.user_id AND m1.prev_month = m2.cur_month
group by m1.cur_month
'''
select(sql)

,cur_month,count(m2.user_id)
0,2021 01,0
1,2021 02,499
2,2021 03,518
3,2021 04,567
4,2021 05,516
5,2021 06,541
6,2021 07,553
7,2021 08,563
8,2021 09,568
9,2021 10,588


## 3.3. Решение Pandas

In [17]:
logins_extended = logins.copy()
logins_extended['id_and_month'] = (
    logins['user_id'].astype(str) 
    + '_' 
    + logins['date'].dt.to_period('M').dt.to_timestamp().astype(str))
logins_extended['id_and_prev_month'] = (
    logins['user_id'].astype(str) 
    + '_' 
    + (logins['date'].dt.to_period('M').dt.to_timestamp() 
       - pd.DateOffset(months=1)).astype(str))
logins_extended = logins_extended.drop_duplicates(['user_id','id_and_month'])
logins_extended

,user_id,date,id_and_month,id_and_prev_month
0,17017,2021-05-21,17017_2021-05-01,17017_2021-04-01
1,14463,2022-03-08,14463_2022-03-01,14463_2022-02-01
2,2059,2021-05-18,2059_2021-05-01,2059_2021-04-01
3,11139,2021-11-03,11139_2021-11-01,11139_2021-10-01
4,17911,2021-08-04,17911_2021-08-01,17911_2021-07-01
...,...,...,...,...
99995,2113,2022-02-17,2113_2022-02-01,2113_2022-01-01
99996,1904,2021-01-29,1904_2021-01-01,1904_2020-12-01
99997,3243,2021-09-06,3243_2021-09-01,3243_2021-08-01
99998,23381,2021-08-21,23381_2021-08-01,23381_2021-07-01


In [18]:
logins_extended['prev_month_login'] = (
    np.where(logins_extended['id_and_prev_month'].isin(logins_extended['id_and_month'])
             , 1 , 0))
logins_extended.groupby([logins_extended['date'].dt.to_period('M')])['prev_month_login'].sum()

<ipython-input-18-51ef2d549242>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  logins_extended['prev_month_login'] = (


date
2021-01      0
2021-02    499
2021-03    518
2021-04    567
2021-05    516
2021-06    541
2021-07    553
2021-08    563
2021-09    568
2021-10    588
2021-11    555
2021-12    565
2022-01    547
2022-02    513
2022-03    530
2022-04    614
2022-05    582
2022-06    541
2022-07    568
2022-08    568
2022-09    591
2022-10    521
2022-11    544
Freq: M, Name: prev_month_login, dtype: int64

# 4. Задача № 4. Нарастающий итог

## 4.0. Поставнока задачи

**Контекст:** допустим, у нас есть таблица transactions в таком виде:

| date       | cash_flow |
|------------|-----------|
| 2018-01-01 | -1000     |
| 2018-01-02 | -100      |
| 2018-01-03 | 50        |
| ...        | ...       |


Где cash_flow — это выручка минус затраты за каждый день.

**Задача:** написать запрос, чтобы получить нарастающий итог для денежного потока каждый день таким образом, чтобы в конечном итоге получилась таблица в такой форме:

| date       | cumulative_cf |
|------------|---------------|
| 2018-01-01 | -1000         |
| 2018-01-02 | -1100         |
| 2018-01-03 | -1050         |
| ...        | ...           |

## 4.1. Подготовка данных

In [26]:
sample_size = 10
start = pd.to_datetime('2021-01-01')

transactions = pd.DataFrame()
transactions['date'] = [start + pd.DateOffset(days=i) for i in range(sample_size)]
transactions['cash_flow'] = np.random.randint(-2000, 2000, sample_size)
transactions

,date,cash_flow
0,2021-01-01,-341
1,2021-01-02,1563
2,2021-01-03,585
3,2021-01-04,-1804
4,2021-01-05,-1733
5,2021-01-06,-619
6,2021-01-07,1454
7,2021-01-08,1496
8,2021-01-09,-353
9,2021-01-10,-1736


In [27]:
transactions.to_sql('transactions', con, if_exists='replace',index=False)

## 4.2. Решение SQL

In [28]:
sql = '''
select
  date,
  sum(cash_flow) over (order by date) AS cumulative_cf
from transactions
group by date
order by date asc
'''
select(sql)

,date,cumulative_cf
0,2021-01-01 00:00:00,-341
1,2021-01-02 00:00:00,1222
2,2021-01-03 00:00:00,1807
3,2021-01-04 00:00:00,3
4,2021-01-05 00:00:00,-1730
5,2021-01-06 00:00:00,-2349
6,2021-01-07 00:00:00,-895
7,2021-01-08 00:00:00,601
8,2021-01-09 00:00:00,248
9,2021-01-10 00:00:00,-1488


## 4.3. Решение SQL без оконной функции

In [29]:
sql = '''
select
  t1.date,
  sum(t2.cash_flow) as cumulative_cf
from transactions as t1
join transactions as t2 on t1.date >= t2.date
group by t1.date
order by t1.date asc
'''
select(sql)

,date,cumulative_cf
0,2021-01-01 00:00:00,-341
1,2021-01-02 00:00:00,1222
2,2021-01-03 00:00:00,1807
3,2021-01-04 00:00:00,3
4,2021-01-05 00:00:00,-1730
5,2021-01-06 00:00:00,-2349
6,2021-01-07 00:00:00,-895
7,2021-01-08 00:00:00,601
8,2021-01-09 00:00:00,248
9,2021-01-10 00:00:00,-1488


## 4.4. Решение Pandas

In [30]:
transactions_extended = transactions.copy().sort_values('date')
transactions_extended['cumulative_cf'] = transactions_extended['cash_flow'].cumsum()
transactions_extended[['date','cumulative_cf']]

,date,cumulative_cf
0,2021-01-01,-341
1,2021-01-02,1222
2,2021-01-03,1807
3,2021-01-04,3
4,2021-01-05,-1730
5,2021-01-06,-2349
6,2021-01-07,-895
7,2021-01-08,601
8,2021-01-09,248
9,2021-01-10,-1488


# 5. Задача № 5. Скользящее среднее

## 5.0. Постановка задачи

**Контекст:** допустим, у нас есть таблица signups в таком виде:

| date       | sign_ups |
|------------|----------|
| 2018-01-01 | 10       |
| 2018-01-02 | 20       |
| 2018-01-03 | 50       |
| ...        | ...      |
| 2018-10-01 | 35       |


**Задача:** написать запрос, чтобы получить 7-дневное скользящее среднее ежедневных регистраций

## 5.1. Подготовка данных

In [31]:
sample_size = 10
start = pd.to_datetime('2021-01-01')

signups = pd.DataFrame()
signups['date'] = [start + pd.DateOffset(days=i) for i in range(sample_size)]
signups['sign_ups'] = np.random.randint(0, 100, sample_size)
signups

,date,sign_ups
0,2021-01-01,69
1,2021-01-02,57
2,2021-01-03,3
3,2021-01-04,56
4,2021-01-05,71
5,2021-01-06,41
6,2021-01-07,84
7,2021-01-08,38
8,2021-01-09,3
9,2021-01-10,33


In [32]:
signups.to_sql('signups', con, if_exists='replace',index=False)

## 5.2. Решение SQL

In [33]:
sql = '''
select
  s1.date,
  s1.sign_ups,
  avg(s2.sign_ups)
from signups as s1

join signups as s2 on s1.date < date(s2.date, '7 days') and s1.date >= s2.date
group by s1.date
'''

results = select(sql)
results

,date,sign_ups,avg(s2.sign_ups)
0,2021-01-01 00:00:00,69,69.000000
1,2021-01-02 00:00:00,57,63.000000
2,2021-01-03 00:00:00,3,43.000000
3,2021-01-04 00:00:00,56,46.250000
4,2021-01-05 00:00:00,71,51.200000
5,2021-01-06 00:00:00,41,49.500000
6,2021-01-07 00:00:00,84,54.428571
7,2021-01-08 00:00:00,38,50.000000
8,2021-01-09 00:00:00,3,42.285714
9,2021-01-10 00:00:00,33,46.571429


## 5.3. Решение Pandas

In [34]:
signups_extended = signups.copy().set_index('date')
signups_extended.rolling('7D').mean()

,sign_ups
date,
2021-01-01,69.000000
2021-01-02,63.000000
2021-01-03,43.000000
2021-01-04,46.250000
2021-01-05,51.200000
2021-01-06,49.500000
2021-01-07,54.428571
2021-01-08,50.000000
2021-01-09,42.285714


# 6. Задача № 6. Несколько условий соединения

## 6.0 Постановка задачи

**Контекст:** скажем, наша таблица emails содержит электронные письма, отправленные с адреса zach@g.com и полученные на него:

| id | subject  | from         | to           | timestamp           |
|----|----------|--------------|--------------|---------------------|
| 1  | Yosemite | zach@g.com   | thomas@g.com | 2018-01-02 12:45:03 |
| 2  | Big Sur  | sarah@g.com  | thomas@g.com | 2018-01-02 16:30:01 |
| 3  | Yosemite | thomas@g.com | zach@g.com   | 2018-01-02 16:35:04 |
| 4  | Running  | jill@g.com   | zach@g.com   | 2018-01-03 08:12:45 |
| 5  | Yosemite | zach@g.com   | thomas@g.com | 2018-01-03 14:02:01 |
| 6  | Yosemite | thomas@g.com | zach@g.com   | 2018-01-03 15:01:05 |
| .. | ..       | ..           | ..           | ..                  |


**Задача:** написать запрос, чтобы получить время отклика на каждое письмо (id), отправленное на zach@g.com. Не включать письма на другие адреса. Предположим, что у каждого треда уникальная тема. Имейте в виду, что в треде может быть несколько писем туда и обратно между zach@g.com и другими адресатами.


## 6.1 Подготовка данных

In [35]:
emails = pd.DataFrame({'subject': ['Yosemite',
                                   'Big Sur',
                                   'Yosemite',
                                   'Running',
                                   'Yosemite',
                                   'Yosemite'],
                       'from' : ['zach@g.com',
                                 'sarah@g.com',
                                 'thomas@g.com',
                                 'jill@g.com',
                                 'zach@g.com',
                                 'thomas@g.com'],
                       'to' : ['thomas@g.com',
                               'thomas@g.com',
                               'zach@g.com',
                               'zach@g.com',
                               'thomas@g.com',
                               'zach@g.com'],
                       'date': [pd.Timestamp('2018-01-02 12:45:03'),
                                pd.Timestamp('2018-01-02 16:30:01'),
                                pd.Timestamp('2018-01-02 16:35:04'),
                                pd.Timestamp('2018-01-03 08:12:45'),
                                pd.Timestamp('2018-01-03 14:02:01'),
                                pd.Timestamp('2018-01-03 15:01:05')]
                       })
emails

,subject,from,to,date
0,Yosemite,zach@g.com,thomas@g.com,2018-01-02 12:45:03
1,Big Sur,sarah@g.com,thomas@g.com,2018-01-02 16:30:01
2,Yosemite,thomas@g.com,zach@g.com,2018-01-02 16:35:04
3,Running,jill@g.com,zach@g.com,2018-01-03 08:12:45
4,Yosemite,zach@g.com,thomas@g.com,2018-01-03 14:02:01
5,Yosemite,thomas@g.com,zach@g.com,2018-01-03 15:01:05


In [36]:
emails.to_sql('emails', con, if_exists='replace')

## 6.2. Решение SQL

In [92]:
sql = '''
select e1.subject,
       e1."from",
       e1."to",
       e1.date,
       e2.date,
       CAST((JulianDay(min(e2.date)) - JulianDay(e1.date)) * 24 as Integer) as diff_in_hours
from emails as e1
join emails as e2 on e1.subject = e2.subject and 
                     e1."from" = e2."to" and
                     e1."to" = e2."from" and
                     e1.date < e2.date
--where e1."to" == "zach@g.com"
group by e1."index"
'''

select(sql)

,subject,from,to,date,date,diff_in_hours
0,Yosemite,zach@g.com,thomas@g.com,2018-01-02 12:45:03,2018-01-02 16:35:04,3
1,Yosemite,thomas@g.com,zach@g.com,2018-01-02 16:35:04,2018-01-03 14:02:01,21
2,Yosemite,zach@g.com,thomas@g.com,2018-01-03 14:02:01,2018-01-03 15:01:05,0


## 6.3. Решение Pandas

In [108]:
emails_extended = pd.merge(emails,
                           emails,
                           how='left',
                           left_on=['subject','from','to'],
                           right_on = ['subject','to','from']
                          ).query('date_x < date_y')
emails_extended['diff_in_hours'] = (emails_extended['date_y'] -
                                    emails_extended['date_x']
                                   )
emails_extended.groupby(['subject',
                         'from_x',
                         'to_x',
                         'date_x'
                        ]).agg({'date_y':'min',
                                'diff_in_hours':'min'
                               }).sort_values(['date_x']).reset_index()

,subject,from_x,to_x,date_x,date_y,diff_in_hours
0,Yosemite,zach@g.com,thomas@g.com,2018-01-02 12:45:03,2018-01-02 16:35:04,0 days 03:50:01
1,Yosemite,thomas@g.com,zach@g.com,2018-01-02 16:35:04,2018-01-03 14:02:01,0 days 21:26:57
2,Yosemite,zach@g.com,thomas@g.com,2018-01-03 14:02:01,2018-01-03 15:01:05,0 days 00:59:04
